In [5]:
import requests
import json
import pickle
import joblib
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# --- Tải các mô hình chuẩn đoán ---

# Mô hình chuẩn đoán tim mạch
with open('Model/ML_heartattack.sav', 'rb') as f:
    doctor = pickle.load(f)
print('Mô hình chuẩn đoán tim mạch (doctor) đã được tải thành công!')

# Mô hình chuẩn đoán trầm cảm
loaded_knn = joblib.load('Model/CDTC_knn.sav')
print("Mô hình KNN cho chuẩn đoán trầm cảm đã được tải thành công!")

# Mô hình chuẩn đoán béo phì NutriAI
NutriAI00, CH = joblib.load('Model/NutriAI.sav')
if NutriAI00:
    print("Mô hình chuẩn đoán béo phì NutriAI đã được tải thành công!")
else:
    print("Lỗi khi tải mô hình chuẩn đoán béo phì NutriAI!")

# --- Lấy dữ liệu từ Firebase ---
firebase_url = "https://bai-test-2ae56-default-rtdb.asia-southeast1.firebasedatabase.app/diagnosis.json"
response = requests.get(firebase_url)
data = response.json()

if data:
    for key, record in data.items():
        print("\nHồ sơ:", key)
        print(record)
        diagnosis_val = record.get("diagnosis", "").lower()
        
        # Xử lý hồ sơ tim mạch có diagnosis bắt đầu bằng "heart1"
        if diagnosis_val.startswith("heart1"):
            try:
                features = [
                    float(record.get("heart_age", 0)),
                    float(record.get("heart_gender", 0)),
                    float(record.get("chest_pain", 0)),
                    float(record.get("blood_pressure", 0)),
                    float(record.get("cholesterol", 0)),
                    float(record.get("heartbeat", 0)),
                    float(record.get("thalassemia", 0))
                ]
                features = np.array(features).reshape(1, -1)
                prediction = doctor.predict(features)
                print("Kết quả chuẩn đoán tim mạch:", prediction[0])
                new_ket_qua = "10" if int(prediction[0]) == 0 else "11"
                update_url = f"https://bai-test-2ae56-default-rtdb.asia-southeast1.firebasedatabase.app/diagnosis/{key}.json"
                payload = {"ket_qua": new_ket_qua}
                update_response = requests.patch(update_url, json=payload)
                if update_response.status_code == 200:
                    print("Cập nhật ket_qua thành công:", new_ket_qua)
                else:
                    print("Lỗi cập nhật ket_qua:", update_response.text)
            except Exception as e:
                print("Lỗi khi chuẩn đoán tim mạch:", e)
        
        # Xử lý hồ sơ trầm cảm có diagnosis bắt đầu bằng "depression2"
        elif diagnosis_val.startswith("depression2"):
            try:
                features = [
                    float(record.get("depress_gender", 0)),
                    float(record.get("depress_age", 0)),
                    float(record.get("ap_luc_hoc_tap", 0)),
                    float(record.get("diem_trung_binh", 0)),
                    float(record.get("muc_do_hai_long", 0)),
                    float(record.get("thoi_gian_ngu", 0)),
                    float(record.get("thoi_quen_an_uong", 0)),
                    float(record.get("su_tu_tu", 0)),
                    float(record.get("so_gio_lam_hoc", 0)),
                    float(record.get("ap_luc_tai_chinh", 0)),
                    float(record.get("tien_su_benh_tam_than", 0))
                ]
                features = np.array(features).reshape(1, -1)
                prediction = loaded_knn.predict(features)
                print("Kết quả chuẩn đoán trầm cảm:", prediction[0])
                new_ket_qua = "20" if int(prediction[0]) == 0 else "21"
                update_url = f"https://bai-test-2ae56-default-rtdb.asia-southeast1.firebasedatabase.app/diagnosis/{key}.json"
                payload = {"ket_qua": new_ket_qua}
                update_response = requests.patch(update_url, json=payload)
                if update_response.status_code == 200:
                    print("Cập nhật ket_qua thành công:", new_ket_qua)
                else:
                    print("Lỗi cập nhật ket_qua:", update_response.text)
            except Exception as e:
                print("Lỗi khi chuẩn đoán trầm cảm:", e)
        
        # Xử lý hồ sơ béo phì có diagnosis là "nutriai"
        elif diagnosis_val == "nutriai":
            try:
                # Thu thập các đặc trưng từ dữ liệu Firebase dựa trên HTML
                features = [
                            float(record.get("gender", 0)),               # Giới tính (vẫn dùng key "obesity_gender")
                            float(record.get("age", 0)),                           # Tuổi: dùng "age" thay vì "obesity_age"
                            float(record.get("height", 0)),                        # Chiều cao: dùng "height" thay vì "obesity_height"
                            float(record.get("weight", 0)),                        # Cân nặng: dùng "weight" thay vì "obesity_weight"
                            float(record.get("family_history_with_overweight", 0)),
                            float(record.get("FAVC", 0)),
                            float(record.get("FCVC", 0)),
                            float(record.get("NCP", 0)),
                            float(record.get("CAEC", 0)),
                            float(record.get("SMOKE", 0)),
                            float(record.get("CH2O", 0)),
                            float(record.get("SCC", 0)),
                            float(record.get("FAF", 0)),
                            float(record.get("TUE", 0)),
                            float(record.get("CALC", 0)),
                            float(record.get("MTRANS", 0))
                        ]
                # Chuyển thành mảng numpy và định dạng lại thành (1, -1)
                features = np.array(features).reshape(1, -1)
                
                # Chuẩn hóa dữ liệu nếu cần (dùng chuan_hoa1 nếu mô hình yêu cầu)
                features1 = CH.transform(features)
                
                # Dự đoán bằng mô hình NutriAI1
                prediction = NutriAI00.predict(features1)
                print("Kết quả chuẩn đoán béo phì:", prediction[0])
                
                # Tạo kết quả theo định dạng yêu cầu: "3" + prediction
                new_ket_qua = "3" + str(int(prediction[0]))
                
                # Cập nhật lại Firebase cho hồ sơ này
                update_url = f"https://bai-test-2ae56-default-rtdb.asia-southeast1.firebasedatabase.app/diagnosis/{key}.json"
                payload = {"ket_qua": new_ket_qua}
                update_response = requests.patch(update_url, json=payload)
                if update_response.status_code == 200:
                    print("Cập nhật ket_qua thành công:", new_ket_qua)
                else:
                    print("Lỗi cập nhật ket_qua:", update_response.text)
            except Exception as e:
                print("Lỗi khi chuẩn đoán béo phì:", e)
        
        else:
            print("Loại chuẩn đoán không xác định.")
else:
    print("Không có dữ liệu hồ sơ.")

Mô hình chuẩn đoán tim mạch (doctor) đã được tải thành công!
Mô hình KNN cho chuẩn đoán trầm cảm đã được tải thành công!
Mô hình chuẩn đoán béo phì NutriAI đã được tải thành công!

Hồ sơ: -OL0CzARNE-9-XZDCK2R
{'age': '21', 'ap_luc_hoc_tap': '3', 'ap_luc_tai_chinh': '3', 'diagnosis': 'depression2', 'diem_trung_binh': '3.3', 'gender': '0', 'ip': '42.118.247.223', 'ket_qua': '20', 'model_choice': 'mlp', 'muc_do_hai_long': '3', 'so_gio_lam_hoc': '4', 'su_tu_tu': '0', 'thoi_gian_ngu': '2', 'thoi_quen_an_uong': '1', 'tien_su_benh_tam_than': '0', 'timestamp': '01:01:22 11/3/2025', 'userName': 'NYN'}
Kết quả chuẩn đoán trầm cảm: 0
Cập nhật ket_qua thành công: 20

Hồ sơ: -OL2IuoryD0JgyIxm1qj
{'age': '21', 'ap_luc_hoc_tap': '2', 'ap_luc_tai_chinh': '3', 'diagnosis': 'depression2', 'diem_trung_binh': '6', 'gender': '1', 'ip': '171.253.243.171', 'ket_qua': '21', 'model_choice': 'mlp', 'muc_do_hai_long': '3', 'so_gio_lam_hoc': '7', 'su_tu_tu': '1', 'thoi_gian_ngu': '2', 'thoi_quen_an_uong': '2', 't